In [1]:
import IPython.display as ipd

import numpy as np
import torch


# import stft_64_pad_0 as stft
import stft_64 as stft
from audio_processing import griffin_lim

from scipy.io.wavfile import read
import time
from reconstruct_functions import *
import random
import os

def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(114514)

Random seed set as 114514


In [2]:
hop_length = 1
win_length = 4 ###at least 4*hop_length, even number
channels = win_length//2+1
full_length = 10000


stft_fn = stft.STFT(filter_length=win_length, hop_length=hop_length, win_length=win_length,
                    window=None)

In [3]:

# def magnitude_to_L_test(magnitude, forward_basis):
#     # magnitude:      [1, 2H+1, T]
#     # forward_basis:  [2H+1, 1, 4H]
    
#     # output L shape: [2H+1,T]
#     H = (magnitude.shape[1]-1)//2
#     basis = np.zeros((2*H+1,2*H+1), dtype= np.float64)
#     basis[:, :] = forward_basis[:,0,:2*H+1]
#     basis_inv = np.linalg.pinv(basis)
#     L = np.matmul(basis_inv, magnitude[0,:,:]**2)
#     L[1:2*H+1] = L[1:2*H+1]/2
#     return L

def compute_reconstruction(input_signal):
    # shape: [1,:]
    magnitude, phase_origin = stft_fn.transform(audio_origin)
    magnitude    = magnitude[:,:,2:-2]
    ans = reconstruct_from_S_with_H_1(magnitude, stft_fn, hop_length=1, frame_number=None)
    if ans[0,0]*audio_origin[0,0]<0:
        ans[:,:] = -ans[:,:]
    return ans


In [4]:
test_time = 5
data_arr = []

for idx in range(test_time):
    audio_origin = (torch.rand((1,full_length), dtype=torch.float64)*2-1)
    ans = compute_reconstruction(audio_origin)
    data_arr.append(torch.mean(torch.abs(audio_origin[0,:] - ans[0,:])))
    print(torch.mean(torch.abs(audio_origin[0,:] - ans[0,:])))

print(np.mean(data_arr))
print(np.var(data_arr))

100%|██████████| 9994/9994 [00:21<00:00, 472.92it/s]


tensor(2.9464e-15, dtype=torch.float64)


100%|██████████| 9994/9994 [00:26<00:00, 381.03it/s]


tensor(7.5918e-15, dtype=torch.float64)


100%|██████████| 9994/9994 [00:25<00:00, 388.50it/s]


tensor(1.9405e-15, dtype=torch.float64)


100%|██████████| 9994/9994 [00:24<00:00, 403.43it/s]


tensor(2.9616e-15, dtype=torch.float64)


100%|██████████| 9994/9994 [00:20<00:00, 485.73it/s]

tensor(3.6977e-15, dtype=torch.float64)
3.827600072829617e-15
3.854678239885183e-30


In [5]:
print(torch.min(audio_origin))

tensor(-0.9999, dtype=torch.float64)
